In [1]:
import psycopg2

In [3]:
# establish a connection to the PostgreSQL database
conn = psycopg2.connect(
    dbname = "dw_cs", 
    user = "postgres", 
    host= 'localhost',
    password = "Mu34zi72",
    port = 5432
)

In [4]:
with conn.cursor() as cur:
    
    cur.execute("SELECT * FROM region;")

    rows = cur.fetchall()

    for row in rows:
        print(row)

(0, 'AFRICA                   ', 'lar deposits. blithely final packages cajole. regular waters are final requests. regular accounts are according to ')
(1, 'AMERICA                  ', 'hs use ironic, even requests. s')
(2, 'ASIA                     ', 'ges. thinly even pinto beans ca')
(3, 'EUROPE                   ', 'ly final courts cajole furiously final excuse')
(4, 'MIDDLE EAST              ', 'uickly special accounts cajole carefully blithely close requests. carefully final asymptotes haggle furiousl')


In [5]:
with conn.cursor() as cur:
    
    cur.execute("SELECT * FROM nation;")

    rows = cur.fetchall()

    for row in rows:
        print(row)

(0, 'ALGERIA                  ', 0, ' haggle. carefully final deposits detect slyly agai')
(1, 'ARGENTINA                ', 1, 'al foxes promise slyly according to the regular accounts. bold requests alon')
(2, 'BRAZIL                   ', 1, 'y alongside of the pending deposits. carefully special packages are about the ironic forges. slyly special ')
(3, 'CANADA                   ', 1, 'eas hang ironic, silent packages. slyly regular packages are furiously over the tithes. fluffily bold')
(4, 'EGYPT                    ', 4, 'y above the carefully unusual theodolites. final dugouts are quickly across the furiously regular d')
(5, 'ETHIOPIA                 ', 0, 'ven packages wake quickly. regu')
(6, 'FRANCE                   ', 3, 'refully final requests. regular, ironi')
(7, 'GERMANY                  ', 3, 'l platelets. regular accounts x-ray: unusual, regular acco')
(8, 'INDIA                    ', 2, 'ss excuses cajole slyly across the packages. deposits print aroun')
(9, 'INDONESIA

In [22]:
def collect_stats(table : str, attributes : list) -> None:
    conn.rollback()
    with conn.cursor() as cur:

        for attribute in attributes:
        
            cur.execute(f"SELECT COUNT(*) FROM {table};")

            all_rows = cur.fetchall()

            cur.execute(f"SELECT COUNT (DISTINCT {attribute}) FROM {table};")

            distinct_rows = cur.fetchall()

            cur.execute(f"SELECT MAX ({attribute}) FROM {table};")
            max = cur.fetchall()

            cur.execute(f"SELECT MIN ({attribute}) FROM {table};")
            min = cur.fetchall()

            print(f'>> Attribute      : {attribute}\n\
   All Rows       : {all_rows}\n\
   Distinct Rows  : {distinct_rows}\n\
   Min            : {min}\n\
   Max            : {max}\n')

In [23]:
a = ["c_custkey", "c_name", "c_address", "c_nationkey", "c_phone", "c_acctbal", "c_mktsegment", "c_comment"]
collect_stats("customer", a)

>> Attribute      : c_custkey
   All Rows       : [(1500000,)]
   Distinct Rows  : [(1500000,)]
   Min            : [(1,)]
   Max            : [(1500000,)]

>> Attribute      : c_name
   All Rows       : [(1500000,)]
   Distinct Rows  : [(1500000,)]
   Min            : [('Customer#000000001',)]
   Max            : [('Customer#001500000',)]

>> Attribute      : c_address
   All Rows       : [(1500000,)]
   Distinct Rows  : [(1500000,)]
   Min            : [('   ,qJqVsHDVWLs6mv6S7Hwh9H',)]
   Max            : [('zzzbtVPaB5eL7AFB07nVjHFMa51j2UMU',)]

>> Attribute      : c_nationkey
   All Rows       : [(1500000,)]
   Distinct Rows  : [(25,)]
   Min            : [(0,)]
   Max            : [(24,)]

>> Attribute      : c_phone
   All Rows       : [(1500000,)]
   Distinct Rows  : [(1499963,)]
   Min            : [('10-100-106-1617',)]
   Max            : [('34-999-998-5763',)]

>> Attribute      : c_acctbal
   All Rows       : [(1500000,)]
   Distinct Rows  : [(818834,)]
   Min            : [

In [24]:
attributes = ["s_suppkey", "s_name", "s_address", "s_nationkey", "s_phone", "s_acctbal", "s_comment"]
collect_stats("supplier", attributes)

>> Attribute      : s_suppkey
   All Rows       : [(100000,)]
   Distinct Rows  : [(100000,)]
   Min            : [(1,)]
   Max            : [(100000,)]

>> Attribute      : s_name
   All Rows       : [(100000,)]
   Distinct Rows  : [(100000,)]
   Min            : [('Supplier#000000001       ',)]
   Max            : [('Supplier#000100000       ',)]

>> Attribute      : s_address
   All Rows       : [(100000,)]
   Distinct Rows  : [(100000,)]
   Min            : [('  , Jd6qNPDAgz',)]
   Max            : [('zzyu4VZw4LGgCMMJG8Yr',)]

>> Attribute      : s_nationkey
   All Rows       : [(100000,)]
   Distinct Rows  : [(25,)]
   Min            : [(0,)]
   Max            : [(24,)]

>> Attribute      : s_phone
   All Rows       : [(100000,)]
   Distinct Rows  : [(100000,)]
   Min            : [('10-100-166-6237',)]
   Max            : [('34-998-900-4911',)]

>> Attribute      : s_acctbal
   All Rows       : [(100000,)]
   Distinct Rows  : [(95588,)]
   Min            : [(Decimal('-999.92'),)]

In [25]:
attributes = ["p_partkey", "p_name", "p_mfgr", "p_brand", "p_type", "p_size", "p_container", "p_retailprice", "p_comment"]
collect_stats("part", attributes)

>> Attribute      : p_partkey
   All Rows       : [(2000000,)]
   Distinct Rows  : [(2000000,)]
   Min            : [(1,)]
   Max            : [(2000000,)]

>> Attribute      : p_name
   All Rows       : [(2000000,)]
   Distinct Rows  : [(1999828,)]
   Min            : [('almond antique aquamarine blanched floral',)]
   Max            : [('yellow white wheat peach aquamarine',)]

>> Attribute      : p_mfgr
   All Rows       : [(2000000,)]
   Distinct Rows  : [(5,)]
   Min            : [('Manufacturer#1           ',)]
   Max            : [('Manufacturer#5           ',)]

>> Attribute      : p_brand
   All Rows       : [(2000000,)]
   Distinct Rows  : [(25,)]
   Min            : [('Brand#11  ',)]
   Max            : [('Brand#55  ',)]

>> Attribute      : p_type
   All Rows       : [(2000000,)]
   Distinct Rows  : [(150,)]
   Min            : [('ECONOMY ANODIZED BRASS',)]
   Max            : [('STANDARD POLISHED TIN',)]

>> Attribute      : p_size
   All Rows       : [(2000000,)]
   Disti

In [26]:
attributes = ["ps_partkey", "ps_suppkey", "ps_availqty", "ps_supplycost", "ps_comment"]
collect_stats("partsupp", attributes)

>> Attribute      : ps_partkey
   All Rows       : [(8000000,)]
   Distinct Rows  : [(2000000,)]
   Min            : [(1,)]
   Max            : [(2000000,)]

>> Attribute      : ps_suppkey
   All Rows       : [(8000000,)]
   Distinct Rows  : [(100000,)]
   Min            : [(1,)]
   Max            : [(100000,)]

>> Attribute      : ps_availqty
   All Rows       : [(8000000,)]
   Distinct Rows  : [(9999,)]
   Min            : [(1,)]
   Max            : [(9999,)]

>> Attribute      : ps_supplycost
   All Rows       : [(8000000,)]
   Distinct Rows  : [(99901,)]
   Min            : [(Decimal('1.0'),)]
   Max            : [(Decimal('1000.0'),)]

>> Attribute      : ps_comment
   All Rows       : [(8000000,)]
   Distinct Rows  : [(7914164,)]
   Min            : [(' Tiresias about the deposits lose bold ideas. furiously final packages sleep quickly after the ironic requests. quickly regular pinto beans cajole slyly furio',)]
   Max            : [('zzle: regular, bold requests hang about the a

In [27]:
attributes = ["l_orderkey", "l_partkey", "l_suppkey", "l_linenumber", "l_quantity", "l_extendedprice", "l_discount", "l_tax", "l_returnflag", "l_linestatus", "l_shipdate", "l_commitdate", "l_receiptdate", "l_shipinstruct", "l_shipmode", "l_comment"]
collect_stats("lineitem", attributes)

>> Attribute      : l_orderkey
   All Rows       : [(59986052,)]
   Distinct Rows  : [(15000000,)]
   Min            : [(1,)]
   Max            : [(60000000,)]

>> Attribute      : l_partkey
   All Rows       : [(59986052,)]
   Distinct Rows  : [(2000000,)]
   Min            : [(1,)]
   Max            : [(2000000,)]

>> Attribute      : l_suppkey
   All Rows       : [(59986052,)]
   Distinct Rows  : [(100000,)]
   Min            : [(1,)]
   Max            : [(100000,)]

>> Attribute      : l_linenumber
   All Rows       : [(59986052,)]
   Distinct Rows  : [(7,)]
   Min            : [(1,)]
   Max            : [(7,)]

>> Attribute      : l_quantity
   All Rows       : [(59986052,)]
   Distinct Rows  : [(50,)]
   Min            : [(Decimal('1'),)]
   Max            : [(Decimal('50'),)]

>> Attribute      : l_extendedprice
   All Rows       : [(59986052,)]
   Distinct Rows  : [(1351462,)]
   Min            : [(Decimal('900.91'),)]
   Max            : [(Decimal('104949.5'),)]

>> Attribute 

ciao